## Setup

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import math
import sklearn
from glob import glob
import tensorflow as tf
from IPython.display import YouTubeVideo

/Users/administrator/miniconda3/envs/dsst_env/lib/python3.6/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [4]:
%pwd

'/Users/administrator/Documents/pex_challenge/analysis_notebooks'

In [5]:
# project directory
project_dir = Path('/Users/administrator/Documents/pex_challenge/')
data_dir = project_dir.joinpath('data/yt8m/frame')

### Load Vocabulary CSV

In [6]:
# get data on the labels for videos
label_file = project_dir.joinpath('data/vocabulary.csv')
# read the csv that contains infromation about labels of videos into dataframe
df_labels = pd.read_csv(label_file.as_posix(), sep=',')

### Generate Weights for each Label

In [34]:
def generate_weights(df):
    '''
    This function interates through the rows of the vocabulary csv
    and it gives a weight to each label on probability that it corresponds to something indoor
    based on the name and definition. It returns a df with an additional column corresponding to weights
    '''
    
    weights = []
    indoor_markers = ['room','office','indoor', 'class', 'house', 'home', 'gym', 'facility', 'building', 'kitchen',
                     'tv', 'computer', 'library', 'cafe', 'restaurant', 'concert', 'guitar', 'pet', 'game',
                     'theatre', 'performance art', 'movie', 'film', 'lunch', 'cook', 'recipe', 'instrument', 'store', 
                     'shop', 'food', 'sing', 'art', 'hair', 'draw', 'anime', 'beauty', 'makeup', 'roof', 'floor',
                     'wall', 'corner', 'party', 'school', 'bake', 'dress']
    outdoor_markers = ['cycling', 'cycle', 'fish', 'fishing', 'transport', 'outdoor', 'outside', 'sun',
                      'sport', 'soccer', 'skate', 'football', 'snow', 'rain', 'wind', 'storm', 'amusement park',
                      'bike', 'weather', 'surf', 'ocean', 'tree', 'sky', 'run', 'farm', 'car', 'nature', 'resort',
                      'pool', 'street', 'ave', 'walk']
    
    #iterate through the dataframe
    for row in df.iterrows():
        weight = 0.5
        for i_marker in indoor_markers:
            if i_marker in str(row[1].Name).lower() or i_marker in str(row[1].WikiDescription).lower():
                weight = 0.8
        for o_marker in outdoor_markers:
            if o_marker in str(row[1].Name).lower() or o_marker in str(row[1].WikiDescription).lower() and weight == 0.5:
                weight = 0.2
        weights.append(weight)
    
    df['weights'] = weights
    return df

In [35]:
generate_weights(df_labels)

,Index,TrainVideoCount,KnowledgeGraphId,Name,WikiUrl,Vertical1,Vertical2,Vertical3,WikiDescription,weights
0,0,788288,/m/03bt1gh,Game,https://en.wikipedia.org/wiki/Game,Games,NaN,NaN,"A game is structured form of play, usually und...",0.8
1,1,539945,/m/01mw1,Video game,https://en.wikipedia.org/wiki/Video_game,Games,NaN,NaN,A video game is an electronic game that involv...,0.8
2,2,415890,/m/07yv9,Vehicle,https://en.wikipedia.org/wiki/Vehicle,Autos & Vehicles,NaN,NaN,A vehicle is a mobile machine that transports ...,0.8
3,3,378135,/m/01jddz,Concert,https://en.wikipedia.org/wiki/Concert,Arts & Entertainment,NaN,NaN,A concert is a live music performance in front...,0.8
4,4,286532,/m/09jwl,Musician,https://en.wikipedia.org/wiki/Musician,Arts & Entertainment,NaN,NaN,A musician is a person who plays a musical ins...,0.8
5,5,236948,/m/0215n,Cartoon,https://en.wikipedia.org/wiki/Cartoon,Arts & Entertainment,NaN,NaN,A cartoon is a type of two-dimensional illustr...,0.2
6,6,203343,/m/01350r,Performance art,https://en.wikipedia.org/wiki/Performance_art,Arts & Entertainment,NaN,NaN,Performance art is a performance presented to ...,0.8
7,7,200813,/m/0k4j,Car,https://en.wikipedia.org/wiki/Car,Autos & Vehicles,NaN,NaN,"A car is a wheeled, self-powered motor vehicle...",0.2
8,8,181579,/m/026bk,Dance,https://en.wikipedia.org/wiki/Dance,Arts & Entertainment,NaN,NaN,Dance is a performance art form consisting of ...,0.8
9,9,156226,/m/0342h,Guitar,https://en.wikipedia.org/wiki/Guitar,Arts & Entertainment,NaN,NaN,The guitar is a musical instrument classified ...,0.8


In [36]:
df_label.to_csv('/')

Index(['Index', 'TrainVideoCount', 'KnowledgeGraphId', 'Name', 'WikiUrl',
       'Vertical1', 'Vertical2', 'Vertical3', 'WikiDescription', 'weights'],
      dtype='object')